In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import string
import os
import requests

from requests import get
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
from xgboost import XGBClassifier
from tqdm import tqdm
from time import sleep

from pathlib import Path
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split, GridSearchCV
from nltk.stem.porter import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from tensorflow.keras import layers, Sequential, callbacks, optimizers, utils, models, applications
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

2023-08-10 19:36:48.184840: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-10 19:36:48.410595: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-10 19:36:48.410717: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-08-10 19:36:48.446589: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-08-10 19:36:49.727450: W tensorflow/stream_executor/platform/de

In [2]:
folder_path = '/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/'
dataframes = []
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)
        dataframes.append(df)
merged_df = pd.concat(dataframes, ignore_index=True)

In [3]:
# action_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/action_df.csv")
# adventure_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/adventure_df.csv")
# animation_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/animation_df.csv")
# biography_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/biography_df.csv")
# comedy_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/comedy_df.csv")
# crime_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/crime_df.csv")
# documentary_df = pd.read_csv("/home/yinghui/code/chooj202/movie_genre_prediction/movie_genre_prediction/raw_data/csv/documentary_df.csv")

# merged_df = pd.concat([action_df, adventure_df, animation_df, biography_df, comedy_df, crime_df, documentary_df])
# merged_df.shape

In [4]:
merged_df.head(5)

,Unnamed: 0,movie,imdb_id,genre,plot,image_url
0,0,It's a Wonderful Life,tt0038650,"['Drama', ' Family', ' Fantasy']",George Bailey has spent his entire life giving...,http://img.omdbapi.com/?i=tt0038650&h=600&apik...
1,1,The Lion King,tt0110357,"['Animation', ' Adventure', ' Drama']",A young lion prince is cast out of his pride b...,http://img.omdbapi.com/?i=tt0110357&h=600&apik...
2,2,Spider-Man: Into the Spider-Verse,tt4633694,"['Animation', ' Action', ' Adventure']","Phil Lord and Christopher Miller, the creative...",http://img.omdbapi.com/?i=tt4633694&h=600&apik...
3,3,Coco,tt2380307,"['Animation', ' Adventure', ' Drama']",Despite his family's baffling generations-old ...,http://img.omdbapi.com/?i=tt2380307&h=600&apik...
4,4,WALL·E,tt0910970,"['Animation', ' Adventure', ' Family']","In a distant, but not so unrealistic, future w...",http://img.omdbapi.com/?i=tt0910970&h=600&apik...


In [5]:
merged_df = merged_df.drop(columns=['Unnamed: 0'])

In [6]:
merged_df = merged_df.drop_duplicates()
merged_df.shape

(8864, 5)

In [7]:
merged_df.sample(5)

,movie,imdb_id,genre,plot,image_url
5042,Stardust,tt0486655,"['Adventure', ' Family', ' Fantasy']",NaN,http://img.omdbapi.com/?i=tt0486655&h=600&apik...
10163,The Broken Circle Breakdown,tt2024519,"['Drama', ' Music', ' Romance']",NaN,http://img.omdbapi.com/?i=tt2024519&h=600&apik...
9497,Flyboys,tt0454824,"['Action', ' Drama', ' History']",NaN,http://img.omdbapi.com/?i=tt0454824&h=600&apik...
9924,Annabelle: Creation,tt5140878,"['Horror', ' Mystery', ' Thriller']",NaN,http://img.omdbapi.com/?i=tt5140878&h=600&apik...
10421,Last Days,tt0403217,"['Drama', ' Music']",NaN,http://img.omdbapi.com/?i=tt0403217&h=600&apik...


In [8]:
merged_df["plot"].notna().sum()

4240

In [9]:
merged_df

,movie,imdb_id,genre,plot,image_url
0,It's a Wonderful Life,tt0038650,"['Drama', ' Family', ' Fantasy']",George Bailey has spent his entire life giving...,http://img.omdbapi.com/?i=tt0038650&h=600&apik...
1,The Lion King,tt0110357,"['Animation', ' Adventure', ' Drama']",A young lion prince is cast out of his pride b...,http://img.omdbapi.com/?i=tt0110357&h=600&apik...
2,Spider-Man: Into the Spider-Verse,tt4633694,"['Animation', ' Action', ' Adventure']","Phil Lord and Christopher Miller, the creative...",http://img.omdbapi.com/?i=tt4633694&h=600&apik...
3,Coco,tt2380307,"['Animation', ' Adventure', ' Drama']",Despite his family's baffling generations-old ...,http://img.omdbapi.com/?i=tt2380307&h=600&apik...
4,WALL·E,tt0910970,"['Animation', ' Adventure', ' Family']","In a distant, but not so unrealistic, future w...",http://img.omdbapi.com/?i=tt0910970&h=600&apik...
...,...,...,...,...,...
21569,The Spy Who Loved Me,tt0076752,"['Action', ' Adventure', ' Thriller']",James Bond is back again and his new mission i...,http://img.omdbapi.com/?i=tt0076752&h=600&apik...
21570,Harold & Kumar Go to White Castle,tt0366551,"['Adventure', ' Comedy']",Harold Lee and Kumar Patel are two stoners who...,http://img.omdbapi.com/?i=tt0366551&h=600&apik...
21571,American Honey,tt3721936,"['Adventure', ' Drama', ' Romance']","Star, a teenage girl with nothing to lose, joi...",http://img.omdbapi.com/?i=tt3721936&h=600&apik...
21574,The Wandering Earth II,tt13539646,"['Action', ' Adventure', ' Drama']",Humans built huge engines on the surface of th...,http://img.omdbapi.com/?i=tt13539646&h=600&api...


In [158]:
merged_df[merged_df["genre"] == "Drama"]

,movie,imdb_id,genre,plot,image_url,clean_plot,clean_title,Action,Adventure,Animation,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western


In [10]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer

In [11]:
def cleaning(sentence):
    if isinstance(sentence, str):
    # Basic cleaning
        sentence = sentence.strip() ## remove whitespaces
        sentence = sentence.lower() ## lowercase 
        sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers

        # Advanced cleaning
        for punctuation in string.punctuation:
            sentence = sentence.replace(punctuation, '') ## remove punctuation

        tokenized_sentence = word_tokenize(sentence) ## tokenize 
        stop_words = set(stopwords.words('english')) ## define stopwords

        tokenized_sentence_cleaned = [ ## remove stopwords
            w for w in tokenized_sentence if not w in stop_words
        ]

        lemmatized = [
            WordNetLemmatizer().lemmatize(word, pos = "v") 
            for word in tokenized_sentence_cleaned
        ]

        cleaned_sentence = ' '.join(word for word in lemmatized)

        return cleaned_sentence

In [12]:
merged_df["clean_plot"] = merged_df["plot"].apply(cleaning)

In [13]:
merged_df["clean_title"] = merged_df["movie"].apply(cleaning)

In [14]:
merged_df.isna().sum()

movie             0
imdb_id           0
genre             0
plot           4624
image_url         0
clean_plot     4624
clean_title       0
dtype: int64

In [15]:
merged_df["genre"] = (
    merged_df["genre"]
    .apply(eval)
    .apply(lambda x: [genre.strip() for genre in x])
    )

In [16]:
merged_df.shape

(8864, 7)

In [17]:
merged_df = merged_df.dropna()

In [18]:
merged_df.shape

(4240, 7)

In [157]:
#check the genre we have
merged_df["genre"].explode().value_counts()

Drama        2633
Comedy       1364
Adventure     948
Action        876
Crime         734
Romance       573
Biography     565
Thriller      495
Horror        466
Mystery       451
Animation     405
Fantasy       347
History       298
Family        289
Music         261
Sci-Fi        236
War           201
Sport         187
Musical       106
Western       105
Film-Noir      59
Name: genre, dtype: int64

In [20]:
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(merged_df["genre"])

# transform target variable
y = multilabel_binarizer.transform(merged_df['genre'])

In [21]:
y.shape

(4240, 21)

In [22]:
genre_names = multilabel_binarizer.classes_

# Adding 
for i in range(len(genre_names)):
    merged_df[f"{genre_names[i]}"] = y[:,i]

merged_df.shape

/tmp/ipykernel_4531/311065743.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df[f"{genre_names[i]}"] = y[:,i]
/tmp/ipykernel_4531/311065743.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df[f"{genre_names[i]}"] = y[:,i]
/tmp/ipykernel_4531/311065743.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

(4240, 28)

In [23]:
merged_df.head(5)

,movie,imdb_id,genre,plot,image_url,clean_plot,clean_title,Action,Adventure,Animation,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,It's a Wonderful Life,tt0038650,"[Drama, Family, Fantasy]",George Bailey has spent his entire life giving...,http://img.omdbapi.com/?i=tt0038650&h=600&apik...,george bailey spend entire life give people be...,wonderful life,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Lion King,tt0110357,"[Animation, Adventure, Drama]",A young lion prince is cast out of his pride b...,http://img.omdbapi.com/?i=tt0110357&h=600&apik...,young lion prince cast pride cruel uncle claim...,lion king,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,Spider-Man: Into the Spider-Verse,tt4633694,"[Animation, Action, Adventure]","Phil Lord and Christopher Miller, the creative...",http://img.omdbapi.com/?i=tt4633694&h=600&apik...,phil lord christopher miller creative mind beh...,spiderman spiderverse,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,Coco,tt2380307,"[Animation, Adventure, Drama]",Despite his family's baffling generations-old ...,http://img.omdbapi.com/?i=tt2380307&h=600&apik...,despite familys baffle generationsold ban musi...,coco,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,WALL·E,tt0910970,"[Animation, Adventure, Family]","In a distant, but not so unrealistic, future w...",http://img.omdbapi.com/?i=tt0910970&h=600&apik...,distant unrealistic future mankind abandon ear...,wall·e,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [31]:
X = merged_df[["clean_title",'clean_plot']]

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [58]:
X_train.shape

(2968, 2)

In [33]:
X_train.head(5)

,clean_title,clean_plot
979,highlander,new york city owner sophisticate antique shop ...
339,dog go heaven,charlie b barkin burt reynolds rascally german...
2594,rider,brady blackburn rodeo bronc rider renown learn...
16460,revenant,night funeral bart soldier kill iraq get grave...
615,cowboy bebop movie,year follow terrorist bomb deadly virus releas...


In [99]:
# Instantiating the TfidfVectorizer
tf_idf_vectorizer = TfidfVectorizer(max_df=0.8, min_df=0.05, max_features=10000)
tf_idf_title_vectorizer = TfidfVectorizer(min_df=10, max_features=10000)
# tf_idf_vectorizer = TfidfVectorizer()

# Training it on the texts
X_train_plot_vec = pd.DataFrame(tf_idf_vectorizer.fit_transform(X_train["clean_plot"]).toarray(),
                 columns = tf_idf_vectorizer.get_feature_names_out())
X_train_title_vec = pd.DataFrame(tf_idf_title_vectorizer.fit_transform(X_train["clean_title"]).toarray(),
                 columns = tf_idf_title_vectorizer.get_feature_names_out())
X_test_plot_vec =  pd.DataFrame(tf_idf_vectorizer.transform(X_test["clean_plot"]).toarray(),
                 columns = tf_idf_vectorizer.get_feature_names_out())
X_test_title_vec =  pd.DataFrame(tf_idf_title_vectorizer.transform(X_test["clean_title"]).toarray(),
                 columns = tf_idf_title_vectorizer.get_feature_names_out())


In [100]:
X_test_title_vec

,adventure,age,american,back,bad,batman,big,black,blue,boy,...,star,story,street,time,two,walk,war,way,wild,world
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.707107,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [101]:
X_train_vec = pd.concat([X_train_plot_vec,X_train_title_vec], axis=1)

In [102]:
X_train_vec

,also,american,another,away,back,base,become,begin,believe,best,...,star,story,street,time,two,walk,war,way,wild,world
0,0.220717,0.0,0.250717,0.0,0.000000,0.0,0.000000,0.000000,0.246153,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.000000,0.0,0.190322,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.188647,0.0,0.156644,0.0,0.000000,0.000000,0.000000,0.185769,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.362578,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.227163,0.262979,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2963,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.349338,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2964,0.326021,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.364683,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2965,0.000000,0.0,0.000000,0.0,0.253672,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2966,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.210618,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [103]:
X_train_vec.shape

(2968, 182)

In [104]:
X_test_vec = pd.concat([X_test_plot_vec,X_test_title_vec], axis=1)

In [105]:
X_train_vec.head(5)

,also,american,another,away,back,base,become,begin,believe,best,...,star,story,street,time,two,walk,war,way,wild,world
0,0.220717,0.0,0.250717,0.0,0.000000,0.0,0.000000,0.000000,0.246153,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.000000,0.0,0.190322,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.188647,0.0,0.156644,0.0,0.000000,0.000000,0.000000,0.185769,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.362578,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.227163,0.262979,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [106]:
X_test_vec.shape

(1272, 182)

In [107]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, accuracy_score

## OneVsRest logistic regression

### Simple OneVsRest logistic regression

In [108]:
log = LogisticRegression()
model_test_1 = OneVsRestClassifier(log)

model_test_1.fit(X_train_vec, y_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [109]:
y_pred = model_test_1.predict(X_test_vec)

In [110]:
accuracy_score(y_test, y_pred)

0.030660377358490566

In [128]:
f1_score(y_test, y_pred, average="micro")

0.39471577261809443

In [111]:
multilabel_binarizer.inverse_transform(y_pred)[1]

('Action', 'Adventure', 'Animation')

In [130]:
merged_df[merged_df.clean_plot == X_test.clean_plot.iloc[1]]

,movie,imdb_id,genre,plot,image_url,clean_plot,clean_title,Action,Adventure,Animation,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
436,The Angry Birds Movie 2,tt6095472,"[Animation, Action, Adventure]",After saving Bird Island from an all-out green...,http://img.omdbapi.com/?i=tt6095472&h=600&apik...,save bird island allout greenpig invasion angr...,angry bird movie,1,1,1,...,0,0,0,0,0,0,0,0,0,0


#### Grouping the topic

In [131]:
from sklearn.decomposition import LatentDirichletAllocation

# Instantiate the LDA 
n_components = 7
lda_model = LatentDirichletAllocation(n_components=n_components, max_iter = 500)

# Fit the LDA on the vectorized documents
lda_model.fit(X_train_vec)

LatentDirichletAllocation(max_iter=500, n_components=7)

In [132]:
X_train_topic_mixture = lda_model.transform(X_train_vec)
X_train_topic_df = pd.DataFrame(X_train_topic_mixture)
X_train_topic_df

,0,1,2,3,4,5,6
0,0.028462,0.028490,0.829393,0.028412,0.028386,0.028392,0.028465
1,0.023153,0.023187,0.728248,0.023152,0.023223,0.155862,0.023175
2,0.027315,0.836026,0.027426,0.027312,0.027251,0.027278,0.027392
3,0.035983,0.783965,0.036180,0.036033,0.035870,0.035899,0.036070
4,0.028464,0.518066,0.028415,0.028336,0.340012,0.028301,0.028406
...,...,...,...,...,...,...,...
2963,0.749841,0.041728,0.041750,0.041680,0.041656,0.041614,0.041730
2964,0.038058,0.772497,0.037930,0.037869,0.037846,0.037844,0.037956
2965,0.029581,0.029617,0.029627,0.282838,0.029582,0.029510,0.569245
2966,0.033263,0.033177,0.801006,0.033119,0.033144,0.033074,0.033218


In [133]:
X_test_topic_mixture = lda_model.transform(X_test_vec)
X_test_topic_df = pd.DataFrame(X_test_topic_mixture)
X_test_topic_df

,0,1,2,3,4,5,6
0,0.040199,0.039913,0.760682,0.039763,0.039769,0.039727,0.039948
1,0.304553,0.269647,0.028008,0.027997,0.313963,0.027867,0.027966
2,0.573644,0.315783,0.022223,0.022100,0.022057,0.022093,0.022101
3,0.029152,0.029157,0.029188,0.328206,0.029076,0.029054,0.526167
4,0.032244,0.544634,0.032278,0.294233,0.032163,0.032147,0.032301
...,...,...,...,...,...,...,...
1267,0.041938,0.042054,0.042045,0.041934,0.041926,0.041865,0.748238
1268,0.036100,0.784082,0.036019,0.036016,0.035895,0.035884,0.036004
1269,0.021169,0.873016,0.021250,0.021189,0.021103,0.021105,0.021168
1270,0.035084,0.034876,0.790863,0.034776,0.034823,0.034740,0.034838


### Training OneVsRest using topic grouping as input

In [50]:
model_test_2 = OneVsRestClassifier(log)
model_test_2.fit(X_train_topic_df, y_train)

NameError: name 'X_train_topic_df' is not defined

In [ ]:
y_pred_2 = model_test_2.predict(X_test_topic_df)
accuracy_score(y_test, y_pred_2)

In [ ]:
multilabel_binarizer.inverse_transform(y_pred_2)[0:10]

In [ ]:
multilabel_binarizer.inverse_transform(y_test)[0:10]

### Using both topic grouping as input + OneVsRest 

In [ ]:
# model_test_3 = OneVsRestClassifier(log)
# model_test_3.fit(X_train_topic_df, y_train)
# model_test_3.fit(X_train_vec, y_train)

In [ ]:
# y_pred_3 = model_test_2.predict(X_test_topic_df)
# accuracy_score(y_test, y_pred_3)

## SGD

In [134]:
sgd = SGDClassifier(random_state = 42)
sgd_model = OneVsRestClassifier(sgd)
sgd_model.fit(X_train_vec,y_train)

OneVsRestClassifier(estimator=SGDClassifier(random_state=42))

In [135]:
y_pred_sgd1 = sgd_model.predict(X_test_vec)

In [136]:
multilabel_binarizer.inverse_transform(y_pred_sgd1)[0:10]

[(),
 ('Adventure', 'Animation'),
 ('Drama',),
 ('Adventure', 'Crime'),
 (),
 ('Drama',),
 ('Drama',),
 ('Drama',),
 ('Drama',),
 ()]

In [137]:
accuracy_score(y_test, y_pred_sgd1)

0.026729559748427674

In [138]:
f1_score(y_test, y_pred_sgd1, average="micro")

0.3696098562628337

In [140]:
multilabel_binarizer.inverse_transform(y_pred_sgd1)[2]

('Drama',)

In [141]:
merged_df[merged_df.clean_plot == X_test.clean_plot.iloc[2]]

,movie,imdb_id,genre,plot,image_url,clean_plot,clean_title,Action,Adventure,Animation,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
693,The Taking of Pelham One Two Three,tt0072251,"[Action, Crime, Thriller]",Four seemingly-unrelated men board subway trai...,http://img.omdbapi.com/?i=tt0072251&h=600&apik...,four seeminglyunrelated men board subway train...,take pelham one two three,1,0,0,...,0,0,0,0,0,0,0,1,0,0


## Random Forest Classifier

In [142]:
rfc = RandomForestClassifier(random_state = 42)
rfc_model = OneVsRestClassifier(rfc)

rfc_model.fit(X_train_vec,y_train)

OneVsRestClassifier(estimator=RandomForestClassifier(random_state=42))

In [143]:
y_pred_rfc1 = rfc_model.predict(X_test_vec)

In [144]:
multilabel_binarizer.inverse_transform(y_pred_rfc1)[0:10]

[('Drama',),
 ('Animation',),
 ('Drama',),
 ('Adventure',),
 (),
 ('Drama',),
 ('Drama',),
 ('Drama',),
 ('Drama',),
 ('Drama',)]

In [145]:
accuracy_score(y_test, y_pred_rfc1)

0.02279874213836478

In [154]:
f1_score(y_test, y_pred_rfc1, average="micro")

0.3510016694490818

In [146]:
multilabel_binarizer.inverse_transform(y_test)[0:10]

[('Action', 'Biography', 'Crime'),
 ('Action', 'Adventure', 'Animation'),
 ('Action', 'Crime', 'Thriller'),
 ('Comedy', 'Horror', 'Sci-Fi'),
 ('Comedy', 'Fantasy', 'Horror'),
 ('Comedy', 'Romance'),
 ('Action', 'Drama', 'History'),
 ('Comedy', 'Crime', 'Family'),
 ('Crime', 'Drama', 'Thriller'),
 ('Thriller', 'War')]

## Poly SVC

In [147]:
X_train1, X_val1, y_train1, y_val1 = train_test_split(merged_df['clean_plot'], y, test_size = 0.3, random_state = 0)
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf1 = tfidf_vectorizer.fit_transform(X_train1)
X_val_tfidf1 = tfidf_vectorizer.transform(X_val1)
sgd = SGDClassifier()
lr = LogisticRegression(C=1.0)
svc = SVC(kernel='poly')
nb = MultinomialNB()
# One vs Restclassifier
model = OneVsRestClassifier(svc)
model.fit(X_train_tfidf1, y_train1)
y_pred1 = model.predict(X_val_tfidf1)
print(f'Accuracy score for SVC poly model is {accuracy_score(y_val1, y_pred1)}')

Accuracy score for SVC poly model is 0.015723270440251572


In [148]:
sgd = SGDClassifier()
lr = LogisticRegression(C=1.0)
svc = SVC(kernel='poly')
nb = MultinomialNB()
# One vs Restclassifier
model = OneVsRestClassifier(svc)
model.fit(X_train_vec, y_train)
y_pred_sgd2 = model.predict(X_test_vec)
print(f'Accuracy score for SVC poly model is {accuracy_score(y_test, y_pred_sgd2)}')

Accuracy score for SVC poly model is 0.020440251572327043


In [149]:
X_train1, X_val1, y_train1, y_val1 = train_test_split(merged_df['clean_plot'], y, test_size = 0.3, random_state = 0)
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf1 = tfidf_vectorizer.fit_transform(X_train1)
X_val_tfidf1 = tfidf_vectorizer.transform(X_val1)
sgd = SGDClassifier()
lr = LogisticRegression(C=1.0)
svc = SVC(kernel='poly')
nb = MultinomialNB()
# One vs Restclassifier
model = OneVsRestClassifier(svc)
model.fit(X_train_tfidf1, y_train1)
y_pred1 = model.predict(X_val_tfidf1)
print(f'Accuracy score for SVC poly model is {accuracy_score(y_val1, y_pred1)}')

Accuracy score for SVC poly model is 0.015723270440251572


## Label Powerset

In [155]:
# using Label Powerset
from skmultilearn.problem_transform import LabelPowerset
# initialize label powerset multi-label classifier
classifier = LabelPowerset(LogisticRegression())
# train
classifier.fit(X_train_vec, y_train)
# predict
predictions_lp = classifier.predict(X_test_vec)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions_lp))
print("\n")

/home/yinghui/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


Accuracy =  0.07861635220125786




## Classifier Chains

In [151]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain

# initialize classifier chains multi-label classifier
classifier = ClassifierChain(LogisticRegression())
# Training logistic regression model on train data
classifier.fit(X_train_vec, y_train)
# predict
predictions = classifier.predict(X_test_vec)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

Accuracy =  0.04874213836477988




In [152]:
multilabel_binarizer.inverse_transform(predictions)[0:3]

[('Drama',), ('Action', 'Adventure', 'Animation'), ('Drama', 'Romance')]

In [153]:
multilabel_binarizer.inverse_transform(y_test)[0:3]

[('Action', 'Biography', 'Crime'),
 ('Action', 'Adventure', 'Animation'),
 ('Action', 'Crime', 'Thriller')]